In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%matplotlib inline
import glob
import matplotlib
import numpy as np
import pandas as pd
import sklearn
import torch
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader,Dataset
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm_notebook as tqdm

In [0]:
data = pd.read_pickle('/content/drive/My Drive/Thesis/kraken_dataset_5_blocks.p')

In [0]:
data

,bids_VWAP,asks_VWAP,midpoint,VWAP_midpoint,fee_avg,fee_min,fee_max,fee_sum,pow_tau,pow_avg,pow_min,pow_max,pow_sum,cval_avg,cval_min,cval_max,cval_sum,txs_count_avg,txs_count_min,txs_count_max,txs_count_sum
timestamp,,,,,,,,,,,,,,,,,,,,,
1569888001275,7813.336316,8555.924248,8308.10,8184.630282,49474630.4,36929713,62894500,247373152,765.326543,876.8,490,1301,4384,8.742982e+11,730362558811,984506603373,4371491237288,2790.0,2429,3221,13950
1569888061272,7779.250110,8566.338188,8307.45,8172.794149,49474630.4,36929713,62894500,247373152,765.326543,876.8,490,1301,4384,8.742982e+11,730362558811,984506603373,4371491237288,2790.0,2429,3221,13950
1569888121272,7804.939444,8561.859796,8300.50,8183.399620,49474630.4,36929713,62894500,247373152,765.326543,876.8,490,1301,4384,8.742982e+11,730362558811,984506603373,4371491237288,2790.0,2429,3221,13950
1569888181272,7781.529613,8555.497684,8291.25,8168.513648,49474630.4,36929713,62894500,247373152,765.326543,876.8,490,1301,4384,8.742982e+11,730362558811,984506603373,4371491237288,2790.0,2429,3221,13950
1569888241277,7778.613401,8550.230853,8266.10,8164.422127,49474630.4,36929713,62894500,247373152,765.326543,876.8,490,1301,4384,8.742982e+11,730362558811,984506603373,4371491237288,2790.0,2429,3221,13950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1580428512514,9213.038111,9733.956196,9506.05,9473.497154,11866502.4,110137,67959596,59332512,345.049626,624.0,13,2592,3120,7.669034e+11,2753530690,15926382462800,3834516992170,1862.0,62,3323,9310
1580428572524,9226.448429,9746.903126,9508.30,9486.675777,11866502.4,110137,67959596,59332512,345.049626,624.0,13,2592,3120,7.669034e+11,2753530690,15926382462800,3834516992170,1862.0,62,3323,9310
1580428632531,9214.750803,9736.947906,9509.55,9475.849354,11639273.2,110137,67959596,58196366,377.857510,648.8,13,2592,3244,7.133957e+11,2753530690,15926382462800,3566978661522,1885.2,62,3323,9426


In [0]:
data.columns

Index(['bids_VWAP', 'asks_VWAP', 'midpoint', 'VWAP_midpoint', 'fee_avg',
       'fee_min', 'fee_max', 'fee_sum', 'pow_tau', 'pow_avg', 'pow_min',
       'pow_max', 'pow_sum', 'cval_avg', 'cval_min', 'cval_max', 'cval_sum',
       'txs_count_avg', 'txs_count_min', 'txs_count_max', 'txs_count_sum'],
      dtype='object')

In [0]:
data = data [['bids_VWAP', 'asks_VWAP', 'midpoint', 'VWAP_midpoint', 'fee_avg',
        'pow_tau', 'pow_avg', 'cval_avg','txs_count_avg']]

In [0]:
data

,bids_VWAP,asks_VWAP,midpoint,VWAP_midpoint,fee_avg,pow_tau,pow_avg,cval_avg,txs_count_avg
timestamp,,,,,,,,,
1569888001275,7813.336316,8555.924248,8308.10,8184.630282,49474630.4,765.326543,876.8,8.742982e+11,2790.0
1569888061272,7779.250110,8566.338188,8307.45,8172.794149,49474630.4,765.326543,876.8,8.742982e+11,2790.0
1569888121272,7804.939444,8561.859796,8300.50,8183.399620,49474630.4,765.326543,876.8,8.742982e+11,2790.0
1569888181272,7781.529613,8555.497684,8291.25,8168.513648,49474630.4,765.326543,876.8,8.742982e+11,2790.0
1569888241277,7778.613401,8550.230853,8266.10,8164.422127,49474630.4,765.326543,876.8,8.742982e+11,2790.0
...,...,...,...,...,...,...,...,...,...
1580428512514,9213.038111,9733.956196,9506.05,9473.497154,11866502.4,345.049626,624.0,7.669034e+11,1862.0
1580428572524,9226.448429,9746.903126,9508.30,9486.675777,11866502.4,345.049626,624.0,7.669034e+11,1862.0
1580428632531,9214.750803,9736.947906,9509.55,9475.849354,11639273.2,377.857510,648.8,7.133957e+11,1885.2


In [0]:
df_train = data[0:18000]

In [0]:
df_val = data[18000:20000]

In [0]:
scaler = MinMaxScaler()
train = scaler.fit_transform(df_train)
val = scaler.transform(df_val)

In [0]:
class My_dataset(Dataset):
    def __init__(self,scaled_data, sequence_lenght):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data = scaled_data
        self.size = len(scaled_data)
        self.sequence_lenght = sequence_lenght

    def __len__(self):
        return (self.size-self.sequence_lenght)

    def __getitem__(self, idx):
        
        features = []
        
        for i in range(self.sequence_lenght):
            values = self.data[idx+i]
            # target = self.data[i+1][3]
            feature = np.delete(values, 3)
            features.append(feature)
            # targets.append(target)
        target = self.data[idx+sequence_lenght][3]
        sample = (torch.Tensor(np.array(features)), torch.Tensor(np.array([target])))
 
        return sample

In [0]:
class My_trend_dataset(Dataset):
    def __init__(self,scaled_data, sequence_lenght):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data = scaled_data
        self.size = len(scaled_data)
        self.sequence_lenght = sequence_lenght

    def __len__(self):
        return (self.size-self.sequence_lenght)

    def __getitem__(self, idx):
        
        features = []
        
        for i in range(self.sequence_lenght):
            values = self.data[idx+i]
            # target = self.data[i+1][3]
            feature = np.delete(values, 3)
            features.append(feature)
            # targets.append(target)
        if self.data[idx+sequence_lenght][3] > self.data[idx+sequence_lenght-1][3]:
            target = 1
        else:
            target = 0
#         target = self.data[idx+sequence_lenght][3]
        sample = (torch.Tensor(np.array(features)), torch.Tensor(np.array([target])))
 
        return sample

In [0]:
class LSTM(nn.Module):
 
    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1,
                 num_layers=2):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
 
        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, batch_first=True)

        self.dropout = nn.Dropout(p=0.1)
 
        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)
 
    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
 
    def forward(self, x):
        lstm_out, self.hidden = self.lstm(x)
        
        # lstm_out = self.dropout(lstm_out[:, -1])
        y_pred = self.linear(lstm_out[:, -1])
        # y_pred = torch.sigmoid(y_pred)
        return y_pred
 

In [0]:
sequence_lenght=20
input_dim = 8
output_dim = 2
hidden_dim = 30
n_layers = 3
bs = 5
num_epochs = 30
learning_rate = 0.01

In [0]:
my_dataset_train = My_trend_dataset(train,sequence_lenght)
my_dataloader_train = DataLoader(my_dataset_train,batch_size=bs,shuffle=True)

In [0]:
my_dataset_val = My_trend_dataset(val,sequence_lenght)
my_dataloader_val = DataLoader(my_dataset_val,batch_size=bs)

In [0]:
model = LSTM(input_dim, hidden_dim, batch_size=bs, output_dim=output_dim, num_layers=n_layers)

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
device

device(type='cuda')

In [0]:
model.to(device)

LSTM(
  (lstm): LSTM(8, 30, num_layers=3, batch_first=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (linear): Linear(in_features=30, out_features=2, bias=True)
)

In [0]:
len(my_dataloader_val)

396

In [0]:
optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)
# loss_fn = torch.nn.L1Loss()
loss_fn = torch.nn.CrossEntropyLoss()
best_loss = 1
 
#####################
# Train model
#####################
 
hist = np.zeros(num_epochs)
 
for t in range(num_epochs):
    model.train()
    train_final_loss = 0.0
    for batch in tqdm(my_dataloader_train, total=len(my_dataloader_train)):
        
        x = batch[0]
        y = batch[1]
        # x = x.type(torch.LongTensor)
        
        x = x.to(device)
        y = y.type(torch.LongTensor)
        y = y.to(device)
        y_pred= model(x)
        train_loss = loss_fn(y_pred, torch.squeeze(y))
        train_final_loss += train_loss
        optimiser.zero_grad()
        train_loss.backward()
        optimiser.step()
    
    train_final_loss = train_loss / len(my_dataloader_train)

    print("Epoch ", t, "train MSE: ", "%.20f" % train_final_loss)
    # hist[t] = loss.item()
 
    # val_loss = 0
    n_correct, n_wrong  = 0, 0
    with torch.no_grad():
      model.eval()
      for batch in tqdm(my_dataloader_val, total=len(my_dataloader_val)):
        x = batch[0]
        y = batch[1]
        # x = x.long()
        # y = y.long()
        x = x.to(device)
        y = y.type(torch.LongTensor)
        y = y.to(device)
        y_pred= model(x)
        y_pred = [torch.argmax(pred).item() for pred in y_pred]
        y = [val.item() for val in y]
        n_correct += np.equal(y_pred,y).sum()
        n_wrong += np.not_equal(y_pred,y).sum()
        # val_loss += loss_fn(y_pred, torch.squeeze(y))
        # print(y_pred, y)
        # print(loss_fn(y_pred, y))

    val_final_loss = n_correct / (n_correct + n_wrong)
    if val_final_loss < best_loss:
      torch.save(model.state_dict(), '/content/drive/My Drive/Thesis/weights.pt')
      best_loss = val_final_loss

    print("Epoch ", t, " val   MSE: ", "%.20f" % val_final_loss)
 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app



Epoch  0 train MSE:  0.00019345649343449622


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Epoch  0  val   MSE:  0.49494949494949497248



Epoch  1 train MSE:  0.00020162649161648005


...


Epoch  10 train MSE:  0.00019837052968796343



Epoch  10  val   MSE:  0.49494949494949497248



Epoch  11 train MSE:  0.00019446737132966518



Epoch  11  val   MSE:  0.49494949494949497248


KeyboardInterrupt: ignored

In [0]:
torch.argmax(y_pred[0])

tensor(0, device='cuda:0', grad_fn=<NotImplemented>)

In [0]:
y_pred[0]

tensor([0.0231, 0.0185], device='cuda:0', grad_fn=<SelectBackward>)

In [0]:
np.equal([torch.argmax(pred).item() for pred in y_pred],[val.item() for val in y]).sum()

3

In [0]:
[val.item() for val in y]

[0, 0, 0, 1, 1]

In [0]:
[torch.argmax(pred).item() for pred in y_pred]

[0, 0, 0, 0, 0]

In [0]:
(y_pred == y).sum()

tensor(0, device='cuda:0')

In [0]:
class Model(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers):
        super(Model, self).__init__()
        self.input_size = input_dim
        self.n_layers = n_layers
        self.hidden_size = hidden_dim
        self.output_size = output_dim
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.n_layers, batch_first=True)
        self.linear = nn.Linear(self.hidden_size, self.output_size)
    def forward(self, x, future=0, y=None):
        outputs = []
        # reset the state of LSTM
        # the state is kept till the end of the sequence
        h_t = torch.zeros(input.size(0), self.hidden_size, dtype=torch.float32)
        c_t = torch.zeros(input.size(0), self.hidden_size, dtype=torch.float32)
        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            h_t, c_t = self.lstm(input_t, (h_t, c_t))
            output = self.linear(h_t)
            outputs += [output]
        for i in range(future):
            if y is not None and random.random() > 0.5:
                output = y[:, [i]]  # teacher forcing
            h_t, c_t = self.lstm(output, (h_t, c_t))
            output = self.linear(h_t)
            outputs += [output]
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class Model(nn.Module):
    def __init__(self, args, data):
        super(Model, self).__init__()
        self.use_cuda = args.cuda
        self.P = args.window;
        self.m = data.m
        self.hidR = args.hidRNN;
        self.hidC = args.hidCNN;
        self.hidS = args.hidSkip;
        self.Ck = args.CNN_kernel;
        self.skip = args.skip;
        self.pt = (self.P - self.Ck)/self.skip
        self.hw = args.highway_window
        self.conv1 = nn.Conv2d(1, self.hidC, kernel_size = (self.Ck, self.m));
        self.GRU1 = nn.GRU(self.hidC, self.hidR);
        self.dropout = nn.Dropout(p = args.dropout);
        if (self.skip > 0):
            self.GRUskip = nn.GRU(self.hidC, self.hidS);
            self.linear1 = nn.Linear(self.hidR + self.skip * self.hidS, self.m);
        else:
            self.linear1 = nn.Linear(self.hidR, self.m);
        if (self.hw > 0):
            self.highway = nn.Linear(self.hw, 1);
            self.output = None;
        if (args.output_fun == 'sigmoid'):
            self.output = F.sigmoid;
        if (args.output_fun == 'tanh'):
            self.output = F.tanh;
    
    def forward(self, x):
        batch_size = x.size(0);
        #CNN
        c = x.view(-1, 1, self.P, self.m);
        c = F.relu(self.conv1(c));
        c = self.dropout(c);
        c = torch.squeeze(c, 3);
        # RNN 
        r = c.permute(2, 0, 1).contiguous();
        _, r = self.GRU1(r);
        r = self.dropout(torch.squeeze(r,0));
        #skip-rnn
        if (self.skip > 0):
            s = c[:,:, int(-self.pt * self.skip):].contiguous();
            s = s.view(batch_size, self.hidC, self.pt, self.skip);
            s = s.permute(2,0,3,1).contiguous();
            s = s.view(self.pt, batch_size * self.skip, self.hidC);
            _, s = self.GRUskip(s);
            s = s.view(batch_size, self.skip * self.hidS);
            s = self.dropout(s);
            r = torch.cat((r,s),1);
            res = self.linear1(r);
        #highway
        if (self.hw > 0):
            z = x[:, -self.hw:, :];
            z = z.permute(0,2,1).contiguous().view(-1, self.hw);
            z = self.highway(z);
            z = z.view(-1,self.m);
            res = res + z;
        if (self.output):
            res = self.output(res);
        return res;

In [0]:
import math
import torch.optim as optim

class Optim(object):

    def _makeOptimizer(self):
        if self.method == 'sgd':
            self.optimizer = optim.SGD(self.params, lr=self.lr)
        elif self.method == 'adagrad':
            self.optimizer = optim.Adagrad(self.params, lr=self.lr)
        elif self.method == 'adadelta':
            self.optimizer = optim.Adadelta(self.params, lr=self.lr)
        elif self.method == 'adam':
            self.optimizer = optim.Adam(self.params, lr=self.lr)
        else:
            raise RuntimeError("Invalid optim method: " + self.method)

    def __init__(self, params, method, lr, max_grad_norm, lr_decay=1, start_decay_at=None):
        self.params = list(params)  # careful: params may be a generator
        self.last_ppl = None
        self.lr = lr
        self.max_grad_norm = max_grad_norm
        self.method = method
        self.lr_decay = lr_decay
        self.start_decay_at = start_decay_at
        self.start_decay = False

        self._makeOptimizer()

    def step(self):
        # Compute gradients norm.
        grad_norm = 0
        for param in self.params:
            grad_norm += math.pow(param.grad.data.norm(), 2)

        grad_norm = math.sqrt(grad_norm)
        if grad_norm > 0:
            shrinkage = self.max_grad_norm / grad_norm
        else:
            shrinkage = 1.

        for param in self.params:
            if shrinkage < 1:
                param.grad.data.mul_(shrinkage)

        self.optimizer.step()
        return grad_norm

    # decay learning rate if val perf does not improve or we hit the start_decay_at limit
    def updateLearningRate(self, ppl, epoch):
        if self.start_decay_at is not None and epoch >= self.start_decay_at:
            self.start_decay = True
        if self.last_ppl is not None and ppl > self.last_ppl:
            self.start_decay = True

        if self.start_decay:
            self.lr = self.lr * self.lr_decay
            print("Decaying learning rate to %g" % self.lr)
        #only decay for one epoch
        self.start_decay = False

        self.last_ppl = ppl

        self._makeOptimizer()

In [0]:
from torch.autograd import Variable


def normal_std(x):
    return x.std() * np.sqrt((len(x) - 1.)/(len(x)))

class Data_utility(object):
    # train and valid is the ratio of training set and validation set. test = 1 - train - valid
    def __init__(self, file_name, train, valid, cuda, horizon, window, normalize = 2):
        self.cuda = cuda;
        self.P = window;
        self.h = horizon
        fin = open(file_name);
        self.rawdat = np.loadtxt(fin,delimiter=',');
        self.dat = np.zeros(self.rawdat.shape);
        self.n, self.m = self.dat.shape;
        self.normalize = 2
        self.scale = np.ones(self.m);
        self._normalized(normalize);
        self._split(int(train * self.n), int((train+valid) * self.n), self.n);
        
        self.scale = torch.from_numpy(self.scale).float();
        tmp = self.test[1] * self.scale.expand(self.test[1].size(0), self.m);
            
        if self.cuda:
            self.scale = self.scale.cuda();
        self.scale = Variable(self.scale);
        
        self.rse = normal_std(tmp);
        self.rae = torch.mean(torch.abs(tmp - torch.mean(tmp)));
    
    def _normalized(self, normalize):
        #normalized by the maximum value of entire matrix.
       
        if (normalize == 0):
            self.dat = self.rawdat
            
        if (normalize == 1):
            self.dat = self.rawdat / np.max(self.rawdat);
            
        #normlized by the maximum value of each row(sensor).
        if (normalize == 2):
            for i in range(self.m):
                self.scale[i] = np.max(np.abs(self.rawdat[:,i]));
                self.dat[:,i] = self.rawdat[:,i] / np.max(np.abs(self.rawdat[:,i]));
            
        
    def _split(self, train, valid, test):
        
        train_set = range(self.P+self.h-1, train);
        valid_set = range(train, valid);
        test_set = range(valid, self.n);
        self.train = self._batchify(train_set, self.h);
        self.valid = self._batchify(valid_set, self.h);
        self.test = self._batchify(test_set, self.h);
        
        
    def _batchify(self, idx_set, horizon):
        
        n = len(idx_set);
        X = torch.zeros((n,self.P,self.m));
        Y = torch.zeros((n,self.m));
        
        for i in range(n):
            end = idx_set[i] - self.h + 1;
            start = end - self.P;
            X[i,:,:] = torch.from_numpy(self.dat[start:end, :]);
            Y[i,:] = torch.from_numpy(self.dat[idx_set[i], :]);

        return [X, Y];

    def get_batches(self, inputs, targets, batch_size, shuffle=True):
        length = len(inputs)
        if shuffle:
            index = torch.randperm(length)
        else:
            index = torch.LongTensor(range(length))
        start_idx = 0
        while (start_idx < length):
            end_idx = min(length, start_idx + batch_size)
            excerpt = index[start_idx:end_idx]
            X = inputs[excerpt]; Y = targets[excerpt];
            if (self.cuda):
                X = X.cuda();
                Y = Y.cuda();  
            yield Variable(X), Variable(Y);
            start_idx += batch_size

In [0]:
import argparse
import math
import time
# import torch
import torch.nn as nn
# from models import LSTNet
# import numpy as np;
import importlib
# from utils import *;
# import Optim

def evaluate(data, X, Y, model, evaluateL2, evaluateL1, batch_size):
    model.eval();
    total_loss = 0;
    total_loss_l1 = 0;
    n_samples = 0;
    predict = None;
    test = None;
    for X, Y in data.get_batches(X, Y, batch_size, False):
        output = model(X);
        if predict is None:
            predict = output;
            test = Y;
        else:
            predict = torch.cat((predict,output));
            test = torch.cat((test, Y));
            scale = data.scale.expand(output.size(0), data.m)
            total_loss += evaluateL2(output * scale, Y * scale).data[0]
            total_loss_l1 += evaluateL1(output * scale, Y * scale).data[0]
            n_samples += (output.size(0) * data.m);
            rse = math.sqrt(total_loss / n_samples)/data.rse
            rae = (total_loss_l1/n_samples)/data.rae
            predict = predict.data.cpu().numpy();
            Ytest = test.data.cpu().numpy();
            sigma_p = (predict).std(axis = 0);
            sigma_g = (Ytest).std(axis = 0);
            mean_p = predict.mean(axis = 0)
            mean_g = Ytest.mean(axis = 0)
            index = (sigma_g!=0);
            correlation = ((predict - mean_p) * (Ytest - mean_g)).mean(axis = 0)/(sigma_p * sigma_g);
            correlation = (correlation[index]).mean();
    return rse, rae, correlation;

def train(data, X, Y, model, criterion, optim, batch_size):
    model.train();
    total_loss = 0;
    n_samples = 0;
    for X, Y in data.get_batches(X, Y, batch_size, True):
        model.zero_grad();
        output = model(X);
        scale = data.scale.expand(output.size(0), data.m)
        loss = criterion(output * scale, Y * scale);
        loss.backward();
        grad_norm = optim.step();
        total_loss += loss.data[0];
        n_samples += (output.size(0) * data.m);
    return total_loss / n_samples

parser = argparse.ArgumentParser(description='PyTorch Time series forecasting')
parser.add_argument('--data', type=str, required=True,
help='location of the data file')
parser.add_argument('--model', type=str, default='LSTNet',
help='')
parser.add_argument('--hidCNN', type=int, default=100,
help='number of CNN hidden units')
parser.add_argument('--hidRNN', type=int, default=100,
help='number of RNN hidden units')
parser.add_argument('--window', type=int, default=24 * 7,
help='window size')
parser.add_argument('--CNN_kernel', type=int, default=6,
help='the kernel size of the CNN layers')
parser.add_argument('--highway_window', type=int, default=24,
help='The window size of the highway component')
parser.add_argument('--clip', type=float, default=10.,
help='gradient clipping')
parser.add_argument('--epochs', type=int, default=100,
help='upper epoch limit')
parser.add_argument('--batch_size', type=int, default=128, metavar='N',
help='batch size')
parser.add_argument('--dropout', type=float, default=0.2,
help='dropout applied to layers (0 = no dropout)')
parser.add_argument('--seed', type=int, default=54321,
help='random seed')
parser.add_argument('--gpu', type=int, default=None)
parser.add_argument('--log_interval', type=int, default=2000, metavar='N',
help='report interval')
parser.add_argument('--save', type=str,  default='model/model.pt',
help='path to save the final model')
parser.add_argument('--cuda', type=str, default=True)
parser.add_argument('--optim', type=str, default='adam')
parser.add_argument('--lr', type=float, default=0.001)
parser.add_argument('--horizon', type=int, default=12)
parser.add_argument('--skip', type=float, default=24)
parser.add_argument('--hidSkip', type=int, default=5)
parser.add_argument('--L1Loss', type=bool, default=True)
parser.add_argument('--normalize', type=int, default=2)
parser.add_argument('--output_fun', type=str, default='sigmoid')
args = parser.parse_args()
args.cuda = args.gpu is not None
if args.cuda:
torch.cuda.set_device(args.gpu)
# Set the random seed manually for reproducibility.
torch.manual_seed(args.seed)
if torch.cuda.is_available():
if not args.cuda:
print("WARNING: You have a CUDA device, so you should probably run with --cuda")
else:
torch.cuda.manual_seed(args.seed)
Data = Data_utility(args.data, 0.6, 0.2, args.cuda, args.horizon, args.window, args.normalize);
print(Data.rse);
model = eval(args.model).Model(args, Data);
if args.cuda:
model.cuda()
nParams = sum([p.nelement() for p in model.parameters()])
print('* number of parameters: %d' % nParams)
if args.L1Loss:
criterion = nn.L1Loss(size_average=False);
else:
criterion = nn.MSELoss(size_average=False);
evaluateL2 = nn.MSELoss(size_average=False);
evaluateL1 = nn.L1Loss(size_average=False)
if args.cuda:
criterion = criterion.cuda()
evaluateL1 = evaluateL1.cuda();
evaluateL2 = evaluateL2.cuda();
best_val = 10000000;
optim = Optim.Optim(
model.parameters(), args.optim, args.lr, args.clip,
)
# At any point you can hit Ctrl + C to break out of training early.
try:
print('begin training');
for epoch in range(1, args.epochs+1):
epoch_start_time = time.time()
train_loss = train(Data, Data.train[0], Data.train[1], model, criterion, optim, args.batch_size)
val_loss, val_rae, val_corr = evaluate(Data, Data.valid[0], Data.valid[1], model, evaluateL2, evaluateL1, args.batch_size);
print('| end of epoch {:3d} | time: {:5.2f}s | train_loss {:5.4f} | valid rse {:5.4f} | valid rae {:5.4f} | valid corr  {:5.4f}'.format(epoch, (time.time() - epoch_start_time), train_loss, val_loss, val_rae, val_corr))
# Save the model if the validation loss is the best we've seen so far.
if val_loss < best_val:
with open(args.save, 'wb') as f:
torch.save(model, f)
best_val = val_loss
if epoch % 5 == 0:
test_acc, test_rae, test_corr  = evaluate(Data, Data.test[0], Data.test[1], model, evaluateL2, evaluateL1, args.batch_size);
print ("test rse {:5.4f} | test rae {:5.4f} | test corr {:5.4f}".format(test_acc, test_rae, test_corr))
except KeyboardInterrupt:
print('-' * 89)
print('Exiting from training early')
# Load the best saved model.
with open(args.save, 'rb') as f:
model = torch.load(f)
test_acc, test_rae, test_corr  = evaluate(Data, Data.test[0], Data.test[1], model, evaluateL2, evaluateL1, args.batch_size);
print ("test rse {:5.4f} | test rae {:5.4f} | test corr {:5.4f}".format(test_acc, test_rae, test_corr))